In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk 
import string 
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
from tqdm import tqdm
from bs4 import BeautifulSoup
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from transformers import pipeline

In [ ]:
df=pd.read_csv('../input/rievew/review_data.csv')
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df=df.dropna()

In [ ]:
import emoji
def deEmojify(inputString):
    cleanedArticle=emoji.demojize(inputString, delimiters=("", ""))
    return cleanedArticle

In [ ]:
df['Text_emo']=df['text'].apply(lambda x:deEmojify(x))

In [ ]:
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

In [ ]:
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)


df['Text_contract']=df['Text_emo'].apply(lambda x:expand_contractions(x))

In [ ]:
def cleaning(Inpdata):
    cleanedArticle1=re.sub(r'\\ruser[?|$|(),"".@#=><|!]Â&*/-',r' ',Inpdata)
    cleanedArticle2=re.sub(r'https?://\S+|www\.\S+',r' ',cleanedArticle1)
    cleanedArticle3=re.sub(r' +', ' ',cleanedArticle2)
    cleanedArticle4=re.sub(r'[^a-z A-Z]',r' ',cleanedArticle3)
    cleanedArticle5=cleanedArticle4.replace("  ", "")
    cleanedArticle6=cleanedArticle5.lower()
    return(cleanedArticle6)

In [ ]:
df['text_clean']=df['Text_contract'].apply(cleaning)

In [ ]:
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

In [ ]:
df["text_lemma"] = df["text_clean"].apply(lambda text: lemmatize_words(text))

In [ ]:
df=df.drop(labels=['text','Text_emo','Text_contract','text_clean'], axis=1)

In [ ]:
pip install happytransformer

In [ ]:
from happytransformer import  HappyTextToText
from happytransformer import TTSettings
happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")
beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=30)

In [ ]:
def Grammar_correct(text):
    text_st="grammar:"+" "+text 
    output_text=happy_tt.generate_text(text_st, args=beam_settings)
    return output_text.text

In [ ]:
df["text_correct"] = df["text_lemma"].apply(Grammar_correct)
df.to_csv('df_correct.csv',index=False)

In [4]:
df=pd.read_csv('../input/df-correct/df_correct.csv')
df.head()


In [18]:
df.isna().sum()

In [6]:
df=df.dropna()

In [17]:
import math
import re
from collections import Counter

WORD = re.compile(r"\w+")


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)


In [22]:
score_list=[]
for index, row in df.iterrows():
    vector1 = text_to_vector(row['text_lemma'])
    vector2 = text_to_vector(row['text_correct'])
    cosine = get_cosine(vector1, vector2)
    score_list.append(np.round(cosine,1))
    

In [26]:
df['sim_score']=score_list
df['Grammar_score']=(df['sim_score']*10).astype('int')
df.head()